# Decision trees

Gli alberi decisionali possono essere usati come modelli di classificazione di regressione

**Implementazione "from scratch" di un albero decisionale per classificazione**



*   Entropia

Per costruire alberi decisionali è prima necessario definire il concetto di entropia. 

L'entropia misura il livello di incertezza legato ai dati in analisi. Preso un set di dati S, etichettati in modo da appartenere a una classe $C_1, ..., C_n$, se tutti i dati appartengono alla stessa classe ci sarà poca incertezza e dunque bassa entropia. Viceversa, se i dati sono distribuiti in tutte le classi in maniera uniforme, ci sarà molta incertezza e una entropia alta. 

Matematicamente, possiamo definite l'entropia di un dataset come:

$H(s) = -p_1*log_2*p_1-...-p_n*log_2*p_n$

dove $p_i$ è la percentuale di dati nella classe $i$




*   Quella che segue è la entropia definita sopra implementata in Python



In [1]:
from typing import List
import math

def entropy(class_probabilities):
    """Given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2)
               for p in class_probabilities
               if p > 0)                     # ignore zero probabilities



*   Vengono ora definite altre due funzioni: la prima calcola a partire da un dataset i cui dati consistono di una tupla(input, etichetta), le probabilità delle classi, la seconda funzione calcola l'entropia di un dataset.



In [5]:
from collections import Counter

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labels):
    return entropy(class_probabilities(labels))



*   L'entropia di una partizione di dati

Negli alberi decisionali, a ogni step decisionale i dati vengono partizionati in dei subset. Sarà quindi utile definire l'entropia di una partizione.

Dato il dataset $S$, se esso viene partizionato in $S_1, ..., S_n$ con percentuali $q_1, ..., q_n$ dei dati, l'entropia è definita come:

$H = q_1*H(S_1), + ... +, q_m*H(S_m)$

L'entropia di una partizione viene implementata nel seguente modo

In [9]:
def partition_entropy(subsets):
    """Returns the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum(data_entropy(subset) * len(subset) / total_count
               for subset in subsets)



*   Caso studio e creazione dell'albero decisionale

*Data science from scratch* prende in esame un dataset di candidati per l'assunzione in una azienda, l'albero decisionale dovrà essere creato (addestrato) a seconda delle caratteristiche dei candidati in relazione alla buona riuscita delle loro interviste.



In [12]:
from typing import NamedTuple, Optional

class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None  # allow unlabeled data

                  #  level     lang     tweets  phd  did_well
inputs = [Candidate('Senior', 'Java',   False, False, False),
          Candidate('Senior', 'Java',   False, True,  False),
          Candidate('Mid',    'Python', False, False, True),
          Candidate('Junior', 'Python', False, False, True),
          Candidate('Junior', 'R',      True,  False, True),
          Candidate('Junior', 'R',      True,  True,  False),
          Candidate('Mid',    'R',      True,  True,  True),
          Candidate('Senior', 'Python', False, False, False),
          Candidate('Senior', 'R',      True,  False, True),
          Candidate('Junior', 'Python', True,  False, True),
          Candidate('Senior', 'Python', True,  True,  True),
          Candidate('Mid',    'Python', False, True,  True),
          Candidate('Mid',    'Java',   True,  False, True),
          Candidate('Junior', 'Python', False, True,  False)
         ]



*   Algoritmo ID3

L'albero decisionale implementerà l'algoritmo ID3, che funziona nella seguente maniera:

1.   Se tutti i dati hanno la stessa etichetta, viene creato un nodo foglia che predice quella etichetta e l'algoritmo termina.
2.   Se la lista degli attributi è vuota (ovvero, non ci sono più domande da fare), viene creato un nodo fogliache predice l'etichetta più frequente e l'algoritmo termina.
3. In alternativa, i dati vengono partizionati su ogni attributo.
4. Viene scelta la partizione con la minore entropia di partizione.
5. Viene aggiunto un nodo decisionale sull'attributo scelto
6. L'algoritmo procede ricorsivamente su ogni subset della partizione sugli attributi rimanenti







* Bisogna ora definire una rappresentazione dell'albero



In [15]:
from typing import NamedTuple, Union, Any

class Leaf(NamedTuple):
    value: Any

class Split(NamedTuple):
    attribute: str
    subtrees: dict
    default_value: Any = None

DecisionTree = Union[Leaf, Split]

hiring_tree = Split('level', {   # First, consider "level".
    'Junior': Split('phd', {     # if level is "Junior", next look at "phd"
        False: Leaf(True),       #   if "phd" is False, predict True
        True: Leaf(False)        #   if "phd" is True, predict False
    }),
    'Mid': Leaf(True),           # if level is "Mid", just predict True
    'Senior': Split('tweets', {  # if level is "Senior", look at "tweets"
        False: Leaf(False),      #   if "tweets" is False, predict False
        True: Leaf(True)         #   if "tweets" is True, predict True
    })
})



*   Funzione per classificare un input



In [24]:
def classify(tree, input):
    """classify the input using the given decision tree"""

    # If this is a leaf node, return its value
    if isinstance(tree, Leaf):
        return tree.value

    # Otherwise this tree consists of an attribute to split on
    # and a dictionary whose keys are values of that attribute
    # and whose values of are subtrees to consider next
    subtree_key = getattr(input, tree.attribute)

    if subtree_key not in tree.subtrees:   # If no subtree for key,
        return tree.default_value          # return the default value.

    subtree = tree.subtrees[subtree_key]   # Choose the appropriate subtree
    return classify(subtree, input)        # and use it to classify the input.



*   Implementazione dell'algoritmo definito sopra



In [25]:
def build_tree_id3(inputs, split_attributes, target_attribute):
    # Count target labels
    label_counts = Counter(getattr(input, target_attribute)
                           for input in inputs)
    most_common_label = label_counts.most_common(1)[0][0]

    # If there's a unique label, predict it
    if len(label_counts) == 1:
        return Leaf(most_common_label)

    # If no split attributes left, return the majority label
    if not split_attributes:
        return Leaf(most_common_label)

    # Otherwise split by the best attribute

    def split_entropy(attribute: str) -> float:
        """Helper function for finding the best attribute"""
        return partition_entropy_by(inputs, attribute, target_attribute)

    best_attribute = min(split_attributes, key=split_entropy)

    partitions = partition_by(inputs, best_attribute)
    new_attributes = [a for a in split_attributes if a != best_attribute]

    # recursively build the subtrees
    subtrees = {attribute_value : build_tree_id3(subset,
                                                 new_attributes,
                                                 target_attribute)
                for attribute_value, subset in partitions.items()}

    return Split(best_attribute, subtrees, default_value=most_common_label)



*   Test dell'albero



In [19]:
tree = build_tree_id3(inputs,
                      ['level', 'lang', 'tweets', 'phd'],
                      'did_well')

Aspettativa: True

In [21]:
classify(tree, Candidate("Junior", "Java", True, False))

True

Aspettativa: False

In [22]:
classify(tree, Candidate("Junior", "Java", True, True))

False

Aspettativa: True

In [23]:
classify(tree, Candidate("Intern", "Java", True, True))

True



**Random forests**

Un problema di questo approccio alla costruzione di un modello di classificazione, è la tendenza all'overfitting. Per rispondere a questo problema si possono utilizzare le cosiddette *random forests*, ovvero un insieme di decision trees diversi tra loro. 

L'output finale in questo caso è una combinazione degli output dei vari alberi; nel caso della classificazione ogni albero esprime un "voto" e l'output finale è l'elemento più votato.



*   Random forests in scikit-learn

Scikit-learn fornisce varie implementazione per i decision trees, di cui una per un classificatore basato su random forests, `RandomForestClassifier`.





*   Creazione di un dataset più ampio e diversificato



In [48]:
                  #  level     lang     tweets  phd  did_well
inputs = [Candidate('Senior', 'Java',   False, False, False),
          Candidate('Senior', 'Java',   False, True,  False),
          Candidate('Mid',    'Python', False, False, True),
          Candidate('Junior', 'Python', False, False, True),
          Candidate('Junior', 'R',      True,  False, True),
          Candidate('Junior', 'R',      True,  True,  False),
          Candidate('Mid',    'R',      True,  True,  True),
          Candidate('Senior', 'Python', False, False, False),
          Candidate('Senior', 'R',      True,  False, True),
          Candidate('Junior', 'Python', True,  False, True),
          Candidate('Senior', 'Python', True,  True,  True),
          Candidate('Mid',    'Python', False, True,  True),
          Candidate('Mid',    'Java',   True,  False, True),
          Candidate('Intern', 'Kotlin', False, True,  False),
          Candidate('Intern', 'R', False, True,  False),
          Candidate('Senior', 'R', True, False,  True),
          Candidate('Mid', 'Java', True, True,  False),
          Candidate('Mid', 'Kotlin', False, False,  True),
          Candidate('Senior', 'Java', True, True,  True),
          Candidate('Junior', 'Python', False, False,  True),
          Candidate('Junior', 'C', False, False,  True),
          Candidate('Senior', 'Python', False, False,  False),
          Candidate('Junior', 'C', True, False,  False)
         ]

In [115]:
import pandas as pd
from sklearn.model_selection import train_test_split

pd_inputs = pd.DataFrame(inputs)
X = pd_inputs.iloc[:, 0:4]
y = pd_inputs.iloc[:, 4]


`RandomForestClassifier` richiede degli attributi di tipo float. Essendo alcune delle variabili del dataset di tipo stringa (categorighe), esse non potranno essere convertite in autoamtico in float. Sarà quindi necessario usare un encoder come `preprocessing.LabelEncoder()`

In [116]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
X_encoded = X.apply(le.fit_transform)

X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size = 1/3, random_state = 1)


*   Test di `RandomForestClassifier`




In [117]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state = 1)
classifier.fit(X_train, y_train)
classifier.score(X_val, y_val)

0.75

Senza modificare gli iperparametri di default si ottiene uno score di 0.75. `n_estimators`, ovvero il numero di alberi nella foresta è di default impostato a 100. La massima profondità di un albero `max_depth` è impostata di default a 2. Modificando questi due iperparametri, ci si può aspettare uno score migliore

In [122]:
classifier = RandomForestClassifier(n_estimators = 1000, max_depth = 5, random_state = 1)
classifier.fit(X_train, y_train)
classifier.score(X_val, y_val)

0.75

Pur aumentando gli iperparametri non aumenta lo score. Un possibile motivo può essere il basso numero di entry nel dataset, che rende il modello overfitting, nonostante sia una Random Forest.